# Question 1 explore and cluster the neighborhoods in Toronto

In [32]:
!pip install geopy
!pip install folium
print("Installed!")

     |████████████████████████████████| 102kB 7.8MB/s ta 0:00:011
Installed!


In [34]:
#importing Libraries
import requests
import lxml.html as lh
import bs4 as bs
import urllib.request
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
import folium

In [2]:
#Getting the data from url
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
res = requests.get(url)
soup = bs.BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0]
df = pd.read_html(str(table))
data = pd.read_json(df[0].to_json(orient='records'))

In [3]:
#First 5 records
data.head()

,Borough,Neighborhood,Postal Code
0,Not assigned,Not assigned,M1A
1,Not assigned,Not assigned,M2A
2,North York,Parkwoods,M3A
3,North York,Victoria Village,M4A
4,Downtown Toronto,"Regent Park, Harbourfront",M5A


In [4]:
#Choosing only data where field Borough doesn't have not assigned value
raw_data_selected = data[data['Borough'] != 'Not assigned']

In [7]:
#Grouping Data
raw_data_selected = raw_data_selected.groupby(['Borough', 'Postal Code'], as_index=False).agg(','.join)

In [6]:
raw_data_selected.head()

,Borough,Neighborhood,Postal Code
2,North York,Parkwoods,M3A
3,North York,Victoria Village,M4A
4,Downtown Toronto,"Regent Park, Harbourfront",M5A
5,North York,"Lawrence Manor, Lawrence Heights",M6A
6,Downtown Toronto,"Queen's Park, Ontario Provincial Government",M7A


In [20]:
raw_data_selected = data[data['Borough'] != 'Not assigned']

In [21]:
raw_data_selected = raw_data_selected.groupby(['Borough', 'Postal Code'], as_index=False).agg(','.join)

In [22]:
raw_data_selected.head()

,Borough,Postal Code,Neighborhood
0,Central Toronto,M4N,Lawrence Park
1,Central Toronto,M4P,Davisville North
2,Central Toronto,M4R,"North Toronto West, Lawrence Park"
3,Central Toronto,M4S,Davisville
4,Central Toronto,M4T,"Moore Park, Summerhill East"


In [24]:
raw_data_selected['Neighborhood'] = np.where(raw_data_selected['Neighborhood'] == 'Not assigned', raw_data_selected['Borough'], raw_data_selected['Neighborhood'])

In [25]:
raw_data_selected.shape

(103, 3)

# Question 2 : Using the Geocoder package or the csv file to create the following dataframe

In [26]:
geospatial_url = "https://cocl.us/Geospatial_data"
geospatial_data = pd.read_csv(geospatial_url)

In [27]:
merged_data = pd.merge(raw_data_selected, geospatial_data, on='Postal Code')

In [28]:
merged_data.head()

,Borough,Postal Code,Neighborhood,Latitude,Longitude
0,Central Toronto,M4N,Lawrence Park,43.728020,-79.388790
1,Central Toronto,M4P,Davisville North,43.712751,-79.390197
2,Central Toronto,M4R,"North Toronto West, Lawrence Park",43.715383,-79.405678
3,Central Toronto,M4S,Davisville,43.704324,-79.388790
4,Central Toronto,M4T,"Moore Park, Summerhill East",43.689574,-79.383160


# Question 3 Explore and cluster the neighborhoods in Toronto

In [35]:
from sklearn.cluster import KMeans
import folium
clustering_data = merged_data.copy()
clustering_data = clustering_data[merged_data.Borough.str.contains("Toronto")]
clustering_data.head()

,Borough,Postal Code,Neighborhood,Latitude,Longitude,Coordinates
0,Central Toronto,M4N,Lawrence Park,43.728020,-79.388790,"(43.7280205, -79.3887901)"
1,Central Toronto,M4P,Davisville North,43.712751,-79.390197,"(43.7127511, -79.3901975)"
2,Central Toronto,M4R,"North Toronto West, Lawrence Park",43.715383,-79.405678,"(43.7153834, -79.40567840000001)"
3,Central Toronto,M4S,Davisville,43.704324,-79.388790,"(43.7043244, -79.3887901)"
4,Central Toronto,M4T,"Moore Park, Summerhill East",43.689574,-79.383160,"(43.6895743, -79.38315990000001)"


In [36]:
toronto_map = folium.Map(location=[43.65, -79.4], zoom_start=12)

X = clustering_data['Latitude']
Y = clustering_data['Longitude']
Z = np.stack((X, Y), axis=1)

kmeans = KMeans(n_clusters=4, random_state=0).fit(Z)

clusters = kmeans.labels_
colors = ['red', 'green', 'blue', 'yellow']
clustering_data['Cluster'] = clusters

for latitude, longitude, borough, cluster in zip(clustering_data['Latitude'], clustering_data['Longitude'], clustering_data['Borough'], clustering_data['Cluster']):
   label = folium.Popup(borough, parse_html=True)
   folium.CircleMarker(
       [latitude, longitude],
       radius=5,
       popup=label,
       color='black',
       fill=True,
       fill_color=colors[cluster],
       fill_opacity=0.7).add_to(toronto_map)  

toronto_map